# Save Images using Depth Camera
This script shows how to capture and save images using a depth camera made by Intel.

In [8]:
# import necessary libraries
import pyrealsense2 as rs
import numpy as np
import cv2
# details of pyrealsens2 library:
# https://intelrealsense.github.io/librealsense/python_docs/_generated/pyrealsense2.html
# detiail of opencv-python library
# https://docs.opencv.org/4.x/

In [9]:
# configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()

In [10]:
# get camera details
pipeline_wrapper = rs.pipeline_wrapper(pipeline)
pipeline_profile = config.resolve(pipeline_wrapper)
device = pipeline_profile.get_device()
device_product_line = str(device.get_info(rs.camera_info.product_line))

In [11]:
# check whether camera is rgb
found_rgb = False
for sensor in device.sensors:
    if sensor.get_info(rs.camera_info.name) == 'RGB Camera':
        found_rgb = True
        break
if found_rgb:
    print("Color sensor exists.")
else:
    print("Color sensor DOES NOT exist.")

Color sensor exists.


In [12]:
# configure resolutions
config.enable_stream(stream_type=rs.stream.depth, 
                     width=640, height=480, 
                     format=rs.format.z16, framerate=30)
# color images
config.enable_stream(stream_type=rs.stream.color, 
                     width=640, height=480, 
                     format=rs.format.bgr8, framerate=30)

In [20]:
# start streaming
pipeline.start(config=config)
instruction = False     # control variable for keyboard input message

try:
    # infinite loop for streaming
    while True:
        # Wait for a set of frames for depth and color images
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()
        if not depth_frame or not color_frame:
            continue
        
        # convert images to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())
        
        # apply colormap on depth images
        # images must be converted to 8-bit per pixel
        # https://shimat.github.io/opencvsharp_docs/html/7633dbe6-e0b8-23bf-6966-dc9b4720e3d1.htm
        # alpha here is a scaling factor applied to the input image
        # any color maps can be used: https://docs.opencv.org/3.4/d3/d50/group__imgproc__colormap.html#gga9a805d8262bcbe273f16be9ea2055a65ab3f207661ddf74511b002b1acda5ec09
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.5), cv2.COLORMAP_JET)
        
        # set color map dimensions
        depth_colormap_dim = depth_colormap.shape
        color_colormap_dim = color_image.shape
        
        # if color image and depth image do not have same dimensions,
        # resize color image to match depth image's dimension
        if depth_colormap_dim != color_colormap_dim:
            resized_color_image = cv2.resize(color_image, dsize=(depth_colormap_dim[1], depth_colormap_dim[0]), interpolation=cv2.INTER_AREA)
            images = np.hstack((resized_color_image, depth_colormap))
        else:
            images = np.hstack((color_image, depth_colormap))
            
        # show images
        cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('RealSense', images)
        # show details about keyboard inputs for user interface
        if not instruction:
            print('Press "q" or "esc" to stop streaming.')
            print('Press "s" to save the image frame.')
            instruction = True
        # wait for a key press (1 ms delay)
        key = cv2.waitKey(1)
        # check keyboard input    
        if key == ord('q') or key == 27:
            break
        elif key == ord('s'):
            print("The image frame is saved.")
        
except:
    print("The streaming is not available.")
    
finally:
    # close all windows
    cv2.destroyAllWindows()
    
    # stop streaming
    pipeline.stop()

Press "q" or "esc" to stop streaming.
Press "s" to save the image frame.
